In [1]:
import os
import json
import numpy as np

import torch
import torch.nn as nn
from torch.optim import AdamW
import cv2
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
import torchvision
from torchvision.transforms import transforms

In [62]:
TRAIN_LABELING_DIR = r"/opt/ml/sy_code/K-Fashion 이미지/Training/라벨링데이터"
VALID_LABELING_DIR = r"/opt/ml/sy_code/K-Fashion 이미지/Validation/라벨링데이터"
TRAIN_IMAGE_DIR = r"/opt/ml/sy_code/K-Fashion 이미지/Training/원천데이터"
VALID_IMAGE_DIR = r"/opt/ml/sy_code/K-Fashion 이미지/Validation/원천데이터"

train_json_files = []
valid_json_files = []

for train_file, valid_file in zip(os.listdir(TRAIN_LABELING_DIR), os.listdir(VALID_LABELING_DIR)):
    train_json_files.append(os.path.join(TRAIN_LABELING_DIR, train_file))
    valid_json_files.append(os.path.join(VALID_LABELING_DIR, valid_file))

print("훈련 데이터 개수:", len(train_json_files))
print("평가 데이터 개수:", len(valid_json_files))

FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/sy_code/K-Fashion 이미지/Training/라벨링데이터'

In [64]:
clothes_class = ["아우터", "하의", "원피스", "상의"]
clothes_xy = {"outer_xy":None, "pants_xy":None, "onepiece_xy":None, "shirt_xy":None}
clothes_fabric = {"outer_fabric":None, "pants_fabric":None, "onepiece_fabric":None, "shirt_fabric":None}

train_image_dict = {}
valid_image_dict = {}

for file in train_json_files:
    with open(file, "rb") as f:
        data = json.load(f)
        file_name = data['이미지 정보']['이미지 파일명']
        for cls, xy, fabric in zip(clothes_class, clothes_xy, clothes_fabric):
            clothes_xy[xy] = data['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'][cls][0].values()
            if not clothes_xy[xy]:
                continue
            try:
                clothes_fabric[fabric] = data['데이터셋 정보']['데이터셋 상세설명']['라벨링'][cls][0]['소재']
            except:
                print(file_name)
                print("--------------------")

    for xy, fabric in zip(clothes_xy.values(), clothes_fabric.values()):
        if not xy:
            continue
        train_image_dict[(file_name, xy)] = fabric[0]

for file in valid_json_files:
    with open(file, "rb") as f:
        data = json.load(f)
        file_name = data['이미지 정보']['이미지 파일명']
        for cls, xy, fabric in zip(clothes_class, clothes_xy, clothes_fabric):
            clothes_xy[xy] = data['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'][cls][0].values()
            if not clothes_xy[xy]:
                continue
            try:
                clothes_fabric[fabric] = data['데이터셋 정보']['데이터셋 상세설명']['라벨링'][cls][0]['소재']
            except:
                print(file_name)
                print("--------------------")

    for xy, fabric in zip(clothes_xy.values(), clothes_fabric.values()):
        if not xy:
            continue
        valid_image_dict[(file_name, xy)] = fabric[0]


@기획샘플@-ops4537k04(2)-바이보늬(샘플,주름멜빵)-얼굴컷,어두운거는 밝게 보정해주시고 치마가 예쁘게 나온걸로 사진 많이 넣어주세여~-IMG_E0624.JPG
--------------------


TypeError: 'NoneType' object is not subscriptable

In [51]:
class ClothDataset(Dataset):
    def __init__(self, image_dict, data_dir=r"/opt/ml/sy_code/원천데이터"):
        """
        Arguments:
            image_dict:Dict[(image_file_name, rect_xy):fabric]
            data_dir:Image path
        """
        super().__init__()
        self.data_dir = data_dir
        self.image_dict = image_dict
        self.image_files = []
        self.dict_label_to_num = {
            "퍼" : 0,
            "니트" : 1,
            "무스탕" : 2,
            "레이스" : 3,
            "스웨이드" : 4,
            "린넨" : 5,
            "앙고라" : 6,
            "메시" : 7,
            "코듀로이" : 8,
            "플리스" : 9,
            "시퀸/글리터" : 10,
            "네오프렌" : 11,
            "데님" : 12,
            "실크" : 13,
            "저지" : 14,
            "스판덱스" : 15,
            "트위드" : 16,
            "자카드" : 17,
            "벨벳" : 18,
            "가죽" : 19,
            "비닐/PVC" : 20,
            "면" : 21,
            "울/캐시미어" : 22,
            "시폰" : 23,
            "합성섬유" : 24,
            "우븐" : 25
        }

        self.setup()

    def setup(self):
        for (file_name, xy), fabric in self.image_dict.items():
            if not fabric:
                continue
            self.image_files.append((os.path.join(self.data_dir, file_name), xy, fabric))

    def __getitem__(self, idx):
        file_name, xy, fabric = self.image_files[idx]

        fabric = fabric[0]
            
        x, y, w, h = map(int, xy)
        im = Image.open(file_name)
        im = transforms.ToTensor()(im)
        im = torchvision.transforms.functional.crop(im, y, x, h, w)
        im = transforms.Resize((300, 300))(im)

        fabric_label = self.dict_label_to_num[fabric]
        return im, fabric_label

    def __len__(self):
        return len(self.image_files)


In [59]:
class ClothModel(nn.Module):
    def __init__(self, num_classes=26):
        super().__init__()
        self.model = resnet18(pretrained=True)
        self.fc = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return self.fc(x)

In [57]:
train_dataset = ClothDataset(train_image_dict)
valid_dataset = ClothDataset(valid_image_dict)

print(len(train_dataset), len(valid_dataset))

NameError: name 'train_image_dict' is not defined

In [ ]:
BATCH_SIZE = 16
EPOCHS = 5
LOG_INTERVAL = 100
LEARNING_RATE = 0.001
SAVE_DIR = "./results"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, drop_last=False, shuffle=False)

model = ClothModel()
model.to(device)

criterion = nn.CrossEntropyLoss()
optim = AdamW(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    model.train()
    loss_value = 0
    matches = 0
    for idx, batch in enumerate(tqdm(train_loader)):
        inputs, labels = batch
 
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()

        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)

        loss.backward()
        optim.step()

        loss_value += loss.item()  # loss값 저장
        matches += (preds == labels).sum().item()  # accuracy값 저장
        if (idx + 1) % LOG_INTERVAL == 0:
            train_loss = loss_value / LOG_INTERVAL
            train_acc = matches / BATCH_SIZE / LOG_INTERVAL
            print(
                f"Epoch[{epoch}/{EPOCHS}]({idx + 1}/{len(train_loader)}) || "
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%}"
            )
            loss_value = 0
            matches = 0

    with torch.no_grad():
        model.eval()
        val_loss_items = []
        val_acc_items = []
        # figure = None
        
        for val_batch in val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)

            # if figure is None:
            #     inputs_np = torch.clone(inputs).detach().cpu().permute(0, 2, 3, 1).numpy()
            #     inputs_np = dataset_module.denormalize_image(inputs_np, dataset.mean, dataset.std)

        val_loss = np.sum(val_loss_items) / len(val_loader)
        val_acc = np.sum(val_acc_items) / len(valid_dataset)
        best_val_loss = min(best_val_loss, val_loss)
        if val_acc > best_val_acc:
            print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
            torch.save(model.module.state_dict(), f"{SAVE_DIR}/best.pth")
            best_val_acc = val_acc
        torch.save(model.module.state_dict(), f"{SAVE_DIR}/last.pth")
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )
        print()

    

 29%|██▊       | 4/14 [00:00<00:00, 12.43it/s]

Epoch[0/5](3/14) || training loss 3.482 || training accuracy 33.33%


 57%|█████▋    | 8/14 [00:00<00:00, 12.65it/s]

Epoch[0/5](6/14) || training loss 7.961 || training accuracy 83.33%


 71%|███████▏  | 10/14 [00:00<00:00, 12.54it/s]

...

Epoch[4/5](6/14) || training loss 2.022 || training accuracy 150.00%


 71%|███████▏  | 10/14 [00:00<00:00, 11.41it/s]

Epoch[4/5](9/14) || training loss 3.267 || training accuracy 225.00%


100%|██████████| 14/14 [00:01<00:00, 11.78it/s]

Epoch[4/5](12/14) || training loss 4.535 || training accuracy 275.00%


In [ ]:
plt.imshow(train_dataset[9][0].permute(1, 2, 0))

NameError: name 'train_dataset' is not defined

In [1]:
"""클래식	매니시	페미닌	히피	모던	컨트리	젠더리스	스포티	레트로	밀리터리
프레피	톰보이	로맨틱	웨스턴	소피스트
케이티드	리조트	 	 	키치/
키덜트	스트리트
 	 	섹시	오리엔탈	아방가르드	 	 	 	힙합	 
 	 	 	 	 	 	 	 	펑크""".split()

['클래식',
 '매니시',
 '페미닌',
 '히피',
 '모던',
 '컨트리',
 '젠더리스',
 '스포티',
 '레트로',
 '밀리터리',
 '프레피',
 '톰보이',
 '로맨틱',
 '웨스턴',
 '소피스트',
 '케이티드',
 '리조트',
 '키치/',
 '키덜트',
 '스트리트',
 '섹시',
 '오리엔탈',
 '아방가르드',
 '힙합',
 '펑크']